# **Create or Update Seafloor Site Metadata File**

This notebook goes through the steps necessary to update or create a new site metadata file.

***
## **Import packages**

In [1]:
from datetime import datetime
import json
import ipywidgets as widgets
from IPython.display import display
import functools
import os

from es_sfgtools.utils.metadata.site import Site, buttons
from es_sfgtools.utils.metadata.utils import start_and_end_dates
from es_sfgtools.utils.archive_pull import download_file_from_archive
from es_sfgtools.utils.metadata.campaign import campaign_checks

***
## **Choosing the appropriate site metadata file**

Using the options below, you may either `create` or `update` a site metadata file. If the site already exists in the earthscope archive, the metadata file should be accessible on https://gage-data.earthscope.org/archive/seafloor. 

Only choose 1 of the 3 options.

### **Option 1:** Create a new site file 

In [ ]:
# ----------------------- Update these values ----------------------- 
# Add site names and networks
names = ['AAAA']                 # list of names used for this site, including the 4 char ID
networks = ['aleutian']          # list of networks this site is a part of              

# Add time of origin (Benchmarks drop date)
time_of_origin = datetime(year=1900, month=1, day=1, 
                          hour=0, minute=0, second=0)

local_geoid_height = 0

# Add array center coordinates
x = 0
y = 0
z = 0

# ---------------------- Do not update code below ---------------------
site_class = Site(names=names, networks=networks, time_of_origin=time_of_origin, local_geoid_height=local_geoid_height, array_center={'x':x, 'y':y, 'z':z})
print("Created site object\n"+ json.dumps(site_class.to_dict(), indent=2))

### **Option 2:** Pull existing site metadata from local file path
Use the cell below to load an existing site metadata file to update

In [ ]:
# Pull in site metadata from json file
# ----------------------- Update these values -----------------------
json_file_path = 'site.json'        # JSON file path

# ---------------------- Do not update code below ---------------------
site_class = import_site(json_file_path)
print(json.dumps(site_class.site, indent=2))

### **Option 3:** Pull existing site metadata directly from GAGE data
Use the cell below to load an existing site metadata file from the GAGE data archive

In [ ]:
# ----------------------- Update these values -----------------------
gage_http_file_link = ''
destination_directory = './gage_data'   # directory where the files will be downloaded to (default is the current directory/gage_data)
token_path = '.'                        # directory where the token will be stored (default is the current directory)


# ---------------------- Do not update code below ---------------------
download_file_from_archive(url=gage_http_file_link, 
                           dest_dir=destination_directory)

file_name = os.path.basename(gage_http_file_link)
site_class = import_site(os.path.join(destination_directory, file_name))
print(json.dumps(site_class.site, indent=2))

## **Optional site additions and updates**
***
Below are optional additions and updates you can make to the site metadata file. You do not need to go in order to update the file. Options include..
* Required site specific metadata
   * Benchmarks 
   * Transponders
* Campaign Data
   * Basic campaign information
   * Surveys
* Other site data
   * Reference Frames

#### **Instructions**
1. Input values in __*Update these Values*__ section
   * For a new section, enter all known values, including a start and end date. If a start and end date is not changed from "1900", then it will not be added into the json
   * For updating an existing section, update only the values you wish to change. Leave all other fields empty or as they are.
2. Run the cell and check the printed JSON input is correct.
3. Use the buttons provided to add or update the campaign.

If you need to see the site class object at any time in the process, scroll to the **Write out JSON file** section at the bottom and run the cell that checks the output. 


In [ ]:
# View current Site file

print(json.dumps(site_class.to_dict(), indent=2))


## **Required site specific metadata**
***

### Benchmarks
Adding multiple benchmarks
1. Add first benchmark by inputing benchmark information
2. Run cell and click "Add New Benchmark" and confirm by viewing the output
3. Enter second benchmark information and rerun the cell to generate widgets
4. Click "Add New Benchmark"
5. Repeat as needed

In [ ]:
benchmark = {'dropPointLocation': {}, 'aPrioriLocation': {}}
# ----------------------- Update these values ----------------------- 
# Benchmark information
benchmark_name = "IVB1-2"                 # "IVB1-1" - REQUIRED
benchmark['benchmarkID'] = ""          # stamped on name

# Enter start and end of benchmark, if only a start date, do not update the end date
benchmark_start = '1900-01-01T00:00:00'
benchmark_end = '1900-01-01T00:00:00'

# Drop points
benchmark['dropPointLocation']['latitude'] = ""         # latitude of benchmark drop point
benchmark['dropPointLocation']['longitude'] = ""        # longitude of benchmark drop point
benchmark['dropPointLocation']['elevation'] = ""        # elevation of benchmark drop point

# Apriori locations  # todo this is a list.. should it be?
benchmark['aPrioriLocation']['latitude'] = ""          # latitude of benchmark drop point
benchmark['aPrioriLocation']['longitude']  = ""        # longitude of benchmark drop point
benchmark['aPrioriLocation']['elevation']  = ""        # elevation of benchmark drop point

# **todo should there be a start and end apriori

# ----------------------- Do not update code below ----------------------- 
benchmark = start_and_end_dates(start_date=benchmark_start, end_date=benchmark_end, dict_to_update=benchmark)

# Button actions
bench_output = widgets.Output()
display(widgets.HBox([buttons['new_benchmark'], buttons['existing_benchmark'], buttons['delete_benchmark']]), bench_output)
buttons['new_benchmark'].on_click(functools.partial(site_class.new_benchmark, benchmark_name, benchmark, bench_output))
buttons['existing_benchmark'].on_click(functools.partial(site_class.update_existing_benchmark, benchmark_name, benchmark, bench_output))
buttons['delete_benchmark'].on_click(functools.partial(site_class.delete_benchmark, benchmark_name, bench_output))

### Benchmark transponder

This section defaults to use the previous benchmark input, if you wish to add the transponder information to a different benchmark, change the default `transponder_benchmark_name`

[Optional]: Add associated battery voltage or extra sensor by changing the `add_sensor` or `add_battery_voltage` booleans to `true`

In [ ]:
transponder = {}
# ----------------------- Update these values ----------------------- 
# Change this if you didn't previously add a benchmark or wish to add to a different benchmark
transponder_benchmark_name = benchmark_name
transponder_address = ""                      # e.g 5209 - REQUIRED

# Other transponder information
transponder['uid'] = ""                       # UID of transponder
transponder['model'] = ""                     # transponder model number
transponder['serialNumber'] = ""              # transponder serial number
transponder['batteryCapacity'] = ""           # e.g 4 Ah
transponder['tat'] = ""                       # TAT in ms

# Optional: Add the start and end dates
transponder_start = '2021-01-01T00:00:00'
transponder_end = '1900-01-01T00:00:00'

# Optional: Add any notable events if applicable
notes = ""

# Optional:  Add battery voltage # TODO can these be added more than once? SHould we just take it out?
add_battery_voltage = False          # set to True if adding a voltage 
voltage_date = '1900-01-01T00:00:00'
voltage_value = 0                    # e.g 12.0

# Optional: Add extra sensor
add_sensor = False                  # set to True if adding a sensor
sensor_type = ""                     # e.g "pressure"
sensor_model_number = ""             # pressure sensor model number
sensor_serial_number = ""            # pressure sensor serial number

# ----------------------- Do not update code below ----------------------- 
if add_battery_voltage:
    print("adding battery voltage")
    transponder['batteryVoltage'] = [{}]
    transponder['batteryVoltage'][0]['date'] = voltage_date.strftime('%Y-%m-%dT%H:%M:%S')
    transponder['batteryVoltage'][0]['voltage'] = voltage_value
    
if add_sensor:
    print("adding sensor")
    transponder['extraSensors'] = [{}]
    transponder['extraSensors'][0]['type'] = sensor_type
    transponder['extraSensors'][0]['model'] = sensor_model_number
    transponder['extraSensors'][0]['serialNumber'] = sensor_serial_number

transponder = start_and_end_dates(start_date=transponder_start, end_date=transponder_end,  dict_to_update=transponder)

# Button actions
tran_output = widgets.Output()
display(widgets.HBox([buttons['new_transponder'], buttons['existing_transponder'], buttons['delete_transponder']]), tran_output)
buttons['new_transponder'].on_click(functools.partial(site_class.new_transponder, transponder_benchmark_name, transponder_address, transponder, tran_output))
buttons['existing_transponder'].on_click(functools.partial(site_class.update_existing_transponder, transponder_benchmark_name, transponder_address, transponder, tran_output))
buttons['delete_transponder'].on_click(functools.partial(site_class.delete_transponder, transponder_benchmark_name, transponder_address, tran_output))

## **Campaign Data**
***
### Campaign information

In [ ]:
campaign = {}
# ----------------------- Update these values ----------------------- 

# Generate the campaign name (Required)
campaign_year = "2024"              # Year of campaign - eg. 2024
campaign_interval = "a"          # alphabetical interval of campaign - 1st: a, 2nd: b, etc..
vessel_code = "1234"                # 4 digit vessel code - references vessel metadata

# Enter information known about the people and vessels involved in the campaign
campaign['type'] = "123"                         # type of campaign: deploy | measure 
campaign['launchVesselName'] = ""             # launch vessel name used in campaign
campaign['recoveryVesselName'] = ""           # recovery vessel name used in campaign
campaign['principalInvestigator'] = ""        # PI name 
campaign['cruiseName'] = ""                   # Name of cruise
campaign['technicianName'] = ""               # technician name
campaign['technicianContact'] = "blah"            # technician contact information (email/phone)

# Enter the start/end date & time of the campaign 
campaign_start = '1900-01-01T00:00:00'
campaign_end = '1900-01-01T00:00:00'

# ----------------------- Do not update code below ----------------------- 
campaign_name, campaign['vesselCode'] = campaign_checks(campaign_year, campaign_interval, vessel_code)
campaign = start_and_end_dates(start_date=campaign_start, end_date=campaign_end, dict_to_update=campaign)

# Button actions 
camp_output = widgets.Output()
display(widgets.HBox([buttons['new_campaign'], buttons['existing_campaign'], buttons['delete_campaign']]), camp_output)
buttons['new_campaign'].on_click(functools.partial(site_class.new_campaign, campaign_name, campaign, camp_output))
buttons['existing_campaign'].on_click(functools.partial(site_class.update_existing_campaign, campaign_name, campaign, camp_output))
buttons['delete_campaign'].on_click(functools.partial(site_class.delete_campaign, campaign_name, camp_output)) # TODO: not working right

### Campaign Survey

The code below assumes the survey was part of an campaign and with a survey vessel entered above. If you wish to add to a different campaign / survey vessel, change the `survey_campaign_name` and `survey['vesselName']` variables from the defaults.

In [ ]:
survey = {}
# ----------------------- Update these values ----------------------- 
 # Change this if you didn't previously add a campaign or wish to add to a different campaign
survey_campaign_name = campaign_name       

# Only Update this value to the survey ID if you wish to update an existing survey, find the survey ID and input (Fromat YYYY_A_GLDR_1)
survey_id = ''                 # campaign_name + interval e.g 2024_A_GLDR_1

survey['pattern'] = ''             # e.g cirle drive | fixed point | mixed
survey['benchmarkIDs'] = []        # List of benchmark IDs in survey['IVB1-1', 'IVB1-2', 'IVB1-3']

# Enter the start and end date/time of the survey
survey_start = '1900-01-01T00:00:00'
survey_end = '1900-01-01T00:00:00'

# Add any notes or commands
survey['commands'] = ''                  # log of commands for a given survey
survey['notes'] = ''                    # i.e any strange data flags

    
# ----------------------- Do not update code below -----------------------    
survey = start_and_end_dates(start_date=survey_start, end_date=survey_end, dict_to_update=survey)

# Button actions
survey_output = widgets.Output()
display(widgets.HBox([buttons['new_survey'], buttons['existing_survey'], buttons['delete_survey']]), survey_output)
buttons['new_survey'].on_click(functools.partial(site_class.new_survey, survey_campaign_name, survey, survey_output))
buttons['existing_survey'].on_click(functools.partial(site_class.update_existing_survey, survey_campaign_name, survey_id, survey, survey_utput))
buttons['delete_survey'].on_click(functools.partial(site_class.delete_survey, survey_campaign_name, survey_id, survey_output))

## Other Site Specific Data
### Reference frames

Add reference frame name and start and end dates used (optional)

In [ ]:
reference_frame = {}
# ----------------------- Update these values ----------------------- 
reference_frame_name = "ITRF2008"         # e.g ITRF2008

# Enter the start/end date & time of the reference frame in ISO string format
ref_start = '1900-01-01T00:00:00'
ref_end = '1900-01-01T00:00:00'

# ---------------------- Do not update code below ---------------------
reference_frame = start_and_end_dates(start_date=ref_start, 
                                      end_date=ref_end, 
                                      dict_to_update=reference_frame)
# Button actions
# Initialize separate output widgets for each button
output_new_ref = widgets.Output()
output_existing_ref = widgets.Output()
output_delete_ref = widgets.Output()

ref_output = widgets.Output()
display(widgets.HBox([buttons['new_ref'], buttons['existing_ref'], buttons['delete_ref']], ref_output))
buttons['new_ref'].on_click(functools.partial(site_class.new_reference_frame, reference_frame_name, reference_frame, ref_output))
buttons['existing_ref'].on_click(functools.partial(site_class.update_existing_reference_frame, reference_frame_name, reference_frame, ref_output))
buttons['delete_ref'].on_click(functools.partial(site_class.delete_reference_frame, rreference_frame_name, ref_output))

# Write out JSON file
***
This step can be completed at any time.

* First check the contents...

In [ ]:
print(json.dumps(site_class.to_dict(), indent=2))

## Export Site to file

In [ ]:
SITE_4_CHAR_ID = "" # 4 char site ID

if not SITE_4_CHAR_ID:
    raise ValueError("Please enter a 4 char site ID")
file_path = f"./{SITE_4_CHAR_ID}.json"                            # Export file path you wish to store
site_class.export_site(file_path)